# Задача о диете.

Нам даны цены и содержание белков, жиров, углеводов в самых популярных продуктах. Требуется найти наиболее сбалансированный ежедневный рацион. Для начала считаем данные.

In [1]:
from scipy.optimize import linprog

import numpy as np
import pandas as pd
sample_1 = pd.read_csv("products.csv")
bounds = pd.read_csv("product_bounds.csv")
sample_1

,Product,Proteins,Fats,Carbohydrates,Callories,Price
0,Шоколад,5.4,35.3,56.5,544,30.0
1,Сникерс,9.3,27.9,54.6,507,40.0
2,Сахар,0.0,0.0,99.0,374,4.0
3,Курица,25.0,7.0,0.0,170,22.0
4,Говядина,28.0,6.8,0.0,170,46.3
5,Свинина,20.0,24.0,0.0,298,22.0
6,Колбаса докторская,12.8,22.2,1.5,257,40.0
7,Колбаса сырокопченая,24.0,43.3,0.0,487,55.0
8,Сосиски молочные,11.0,23.9,1.6,266,56.9
9,Морская капуста,0.9,0.2,3.5,12,18.0


А также нам даны границы потребления белков жиров и углеводов.

In [2]:
bounds

,Name,Up_bound,Down_bound
0,Белки,130,90
1,Жиры,120,80
2,Углеводы,250,220
3,Каллории,2300,2000


В терминах линейного программирования исходная задача выглядит как:
$$\langle Price, \vec{x}\rangle \rightarrow \min$$
$$K \cdot \vec{x} \leqslant Up\_bound,$$
$$K \cdot \vec{x} \geqslant Down\_bound,$$
$$\vec{x} \geqslant 0,$$

где $K \in Matr_{\mathbb{R}}(4 \times 67), \vec{x}\in \mathbb{R}^{67}.$

Избавимся от второго неравенства, домножив его на $-1,$ а затем припишем к первому неравенству.

In [3]:
bound = np.array(list(np.array(bounds.Up_bound)) + list(-np.array(bounds.Down_bound)))
K = np.matrix([sample_1.Proteins, sample_1.Fats, sample_1.Carbohydrates, sample_1.Callories])
D = np.vstack((np.array(K),np.array(-K)))
bnds = [(0, None) for _ in range(len(sample_1))]
c = np.array(sample_1.Price)

Теперь задача имеет вид: $$\langle Price, \vec{x}\rangle \rightarrow \min$$
$$D \cdot \vec{x} \leqslant bound,$$
$$\vec{x} \geqslant 0.$$

Наша задача относится к задачам линейного программирования. Для ее решения удобно использовать симплекс-метод с последовательным улучшением плана, так как он является основным методом решения задач такого вида и реализован в библиотеке SciPy.

In [4]:
res = linprog(c, A_ub=D, b_ub=bound, bounds=bnds)
print(res)

  status: 0
   slack: array([  40.        ,   40.        ,    7.19132757,  300.        ,
          0.        ,    0.        ,   22.80867243,    0.        ])
 success: True
     fun: 43.472645808856143
       x: array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.87178035,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.75317671,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  3.30562987,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.       

Напечатаем результат в более удобной форме.

In [5]:
param = np.array(np.zeros(5))
for i in zip(res.x, sample_1.Product, sample_1.Proteins,  sample_1.Fats, \
             sample_1.Carbohydrates, sample_1.Callories, sample_1.Price):
    if i[0] > 0:
        param += i[0] * np.array(i[2:])
        print i[1], '*', round(i[0],3)*100 , "гр"
print "Total:",round(param[0],1),"proteins",round(param[1],1),"fats,",round(param[2],1),"carb.,",\
    round(param[3],1),"callories,",round(param[4],1),"rubles."

Мука пшеничная * 87.2 гр
Масло растительное * 75.3 гр
Чечевица * 330.6 гр
Total: 90.0 proteins 80.0 fats, 242.8 carb., 2000.0 callories, 43.5 rubles.


Получившийся набор весьма скудный, зато стоит всего лишь 44 рубля! Ограничим значения растительного масла и майонеза. Пересчитаем решение. 

In [6]:
bnds[35] = (0, 0.4)
bnds[33] = (0,0.3)
res = linprog(c, A_ub=D, b_ub=bound, bounds=bnds)

In [7]:
param = np.array(np.zeros(5))
for i in zip(res.x, sample_1.Product, sample_1.Proteins,  sample_1.Fats, \
             sample_1.Carbohydrates, sample_1.Callories, sample_1.Price):
    if i[0] > 0:
        param += i[0] * np.array(i[2:])
        print i[1], '*', round(i[0],3)*100 , "гр"
print "Total:",round(param[0],1),"proteins",round(param[1],1),"fats,",round(param[2],1),"carb.,",\
    round(param[3],1),"callories,",round(param[4],1),"rubles."

Сельдь * 77.7 гр
Макароны * 150.8 гр
Майонез * 30.0 гр
Масло растительное * 40.0 гр
Чечевица * 240.5 гр
Total: 90.0 proteins 80.0 fats, 237.3 carb., 2000.0 callories, 47.7 rubles.


Данный набор более близок к реальности. Усложним задачу, добавив границы потребления витаминов A,B,C,D,E,K. 

### Подсчет с учетом витаминов.

In [8]:
sample_2 = pd.read_csv("products_2.csv")
bounds_2 = pd.read_csv("product_bounds_2.csv")
bound_2 = np.array(list(np.array(bounds_2.Up_bound)) + list(-np.array(bounds_2.Down_bound)))
K_2 = np.matrix([sample_2.Proteins, sample_2.Fats, sample_2.Carbohydrates, sample_2.Callories,\
                 sample_2.A, sample_2.B, sample_2.C, sample_2.D, sample_2.E, sample_2.K])
D_2 = np.vstack((np.array(K_2),np.array(-K_2)))

In [9]:
bnds_2 = [(0, None) for _ in range(len(sample_2))]
bnds_2[35] = (0, 0.4)
bnds_2[33] = (0,0.3)
c = np.array(sample_2.Price)

Пересчитаем результат:

In [10]:
res = linprog(c, A_ub=D_2, b_ub=bound_2, bounds=bnds_2)

In [11]:
param = np.array(np.zeros(11))
for i in zip(res.x, sample_1.Product, sample_2.Proteins,  sample_2.Fats,\
             sample_2.Carbohydrates, sample_2.Callories, sample_2.Price,\
             sample_2.A,sample_2.B,sample_2.C,sample_2.D,sample_2.E,sample_2.K):
    if i[0] > 0:
        param += i[0] * np.array(i[2:])
        print i[1], '*', round(i[0],3)*100 , "гр"
print "\nTotal:",round(param[4],1),"rubles\n",param[0],"proteins\n", round(param[1],1),"fats\n",\
    round(param[2],1),"carb.\n",\
    round(param[3],1),"callories\n",\
    round(param[5],1), "A\n",round(param[6],1),"B\n",\
    round(param[7],1),"C\n",round(param[8],1),"D\n",\
    round(param[9],1),"E\n",round(param[10],1),"K\n"

Окунь речной * 112.8 гр
Сельдь * 166.7 гр
Макароны * 305.6 гр
Масло растительное * 40.0 гр
Овсяная каша * 34.5 гр
Капуста белокочанная * 208.6 гр
Ананас * 0.5 гр

Total: 59.3 rubles
90.0 proteins
80.0 fats
250.0 carb.
2082.2 callories
0.7 A
40.0 B
100.0 C
0.1 D
27.2 E
1.5 K



##Вывод
Итак, после учета витаминов последний набор наиболее близок к реальности. Благодаря этому проекту я поняла, что симплекс метод является достаточно быстрым, удобным и надежным методом минимизации линейных функций.